In [ ]:
#%pip install transformers
#%pip install sentence-transformers

In [ ]:
#%pip3 install pytorch torchvision

In [1]:
##imports##
import pandas as pd
import numpy as np
import json
import os
from random import sample 
from numpyencoder import NumpyEncoder
import math
from unicodedata import normalize
from datetime import datetime  
from datetime import timedelta  
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from scipy.sparse import csr_matrix

# NLTK Stop words
import nltk
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
import operator
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.keyedvectors import KeyedVectors
# spacy for lemmatization
import spacy
from spacy.lang.es import Spanish
stop_words = stopwords.words('spanish')
stop_words.extend(['i','ii','iii','iv','v','vi','vii', 'ix' , 'x' , 'xi'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aleja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nlp  =  spacy.load("es_core_news_sm")

C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


# Test con Bert

Vamos a usar el modelo pre entrenado BERT para calcular los vectores con word embeddings

In [3]:
# BERT (PRUEBA)

# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 

# Define the model repo
model_name = "dccuchile/bert-base-spanish-wwm-uncased" 

# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
    

C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ejemplo de uso:

Primero se tokeniza el documento con el "tokenizador" pre entrenado de BERT. Después se aplica el modelo pre entrenado de word embedding a la frase "tokenizada"

In [ ]:
# Transform input tokens 
inputs = tokenizer("Hello world!", return_tensors="pt")
print(inputs)

# Model apply
outputs = model(**inputs)
print(outputs)

In [10]:
# Algunas Funciones útiles

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc = True,max_len = 22))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc),max_len = 22,deacc = True) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV','PROPN','X']):
    """https://spacy.io/api/annotation"""
    texts_out  =  []
    for sent in texts:
        doc  =  nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def make_agg_vec(words, model, num_features, model_word_set, filter_out = []):
    """Create aggregate representation of list of words"""
    feature_vec  =  np.zeros((num_features,), dtype = "float32")
    nwords  =  0.0
    lost_words = []
    for word in words:
        if word not in filter_out:
            if word in model_word_set:
                nwords += 1
                feature_vec = np.add(feature_vec , model[word])
            else:
                lost_words.append(word)
    if nwords>0:
        avg_feature_vec = feature_vec / nwords
        return [avg_feature_vec,lost_words]
    else:
      return [feature_vec,lost_words]


def cos_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def cos_similarity_distance(v1, v2):
 return 1-cos_similarity(v1,v2)


def cos_similarity_angle(v1, v2):
    prod  =  np.dot(v1, v2)
    len1  =  math.sqrt(np.dot(v1, v1))
    len2  =  math.sqrt(np.dot(v2, v2))
    if   abs(prod / (len1 * len2) -1)<(10**(-4)):
        return 0
    if len1*len2 == 0:
        return -1
    else:
      return   math.degrees(    math.acos( prod / (len1 * len2))  )# retorna un angulo entre 0 y 180
    


def cos_similarity_pearson(v1, v2):
    v1    = v1-np.mean(v1)
    v2    = v2-np.mean(v2)
    prod  =  np.dot(v1, v2)
    len1  =  math.sqrt(np.dot(v1, v1))
    len2  =  math.sqrt(np.dot(v2, v2))
    if len1*len2 == 0:
        return -1
    else:
      return   prod / (len1 * len2)  # retorna un angulo entre 0 y 180
    


def preprocess_document(doc):
    """ Preprocesar el documento eliminando stopwords, lematizando, eliminando caracteres no alfanuméricos y palabras vacías. """
    # Asegurarse de que el documento sea una cadena
    if isinstance(doc, list):
        doc = ' '.join(doc)
    # Eliminar stopwords y lematizar
    # doc = nlp(doc)
    # lemmatized = [token.lemma_ for token in doc if not token.is_stop]
    # print(doc)
    no_stopw=remove_stopwords([doc])
    # print(no_stopw)
    lemmatized=lemmatization(no_stopw)
    # Eliminar caracteres no alfanuméricos, convertir a minúsculas y eliminar palabras vacías
    # cleaned = [re.sub(r'\W+', '', word).lower() for word in lemmatized]
    cleaned = [word for word in lemmatized]  # Elimina las palabras vacías
    cleaned=cleaned[0]
    return cleaned


In [18]:
import torch

def make_bert_vec(doc, model, tokenizer):
    """Generate BERT embedding for a single document."""
    encoded_input = tokenizer(doc, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)
    
    # Promedio de todos los tokens para obtener un vector único para el documento
    mean_embedding = output.last_hidden_state.mean(dim=1).squeeze()
    return mean_embedding.numpy()

# Ejemplo de uso
testmessage = "hola, me llamo antonio y estoy ansioso por comenzar el curso"
bert_vector = make_bert_vec(preprocess_document(testmessage), model, tokenizer)


def make_bert_cls_vec(doc, model, tokenizer):
    """Generate the BERT [CLS] embedding for a single document.
       
       Para utilizar el vector [CLS] de BERT para tareas de similitud, modificaremos la función de generación de vectores 
       para que devuelva específicamente el vector [CLS] en lugar del promedio de todos los vectores de token. El token [CLS] 
       se encuentra al principio de cada secuencia y se entrena para representar el significado agregado de toda la entrada, 
       lo que lo hace útil para tareas de comparación de documentos."""
    
    # Tokenizar el documento
    encoded_input = tokenizer(doc, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Pasar el documento tokenizado a través del modelo BERT
    with torch.no_grad():
        output = model(**encoded_input)

    # Obtener el vector [CLS] (primera posición del tensor de salida)
    cls_embedding = output.last_hidden_state[:, 0, :].squeeze()
    return cls_embedding.numpy()

# Ejemplo de uso
testmessage = "hola, me llamo antonio y estoy ansioso por comenzar el curso"
bert_cls_vector = make_bert_cls_vec(testmessage, model, tokenizer)


Vamos a usar los mismos diccionarios y documentos que en el ejemplo con FastText

In [21]:
# Diccionarios

content_test_dictionary = [
    "física",
    "energía",
    "átomo",
    "mecánica",
    "cuántica",
    "electromagnetismo",
    "termodinámica",
    "relatividad",
    "partículas",
    "óptica",
    "nuclear",
    "experimento",
    "teoría",
    "leyes",
    "fórmula",
    "ondas",
    "electrones",
    "gravitación",
    "espectro",
    "campo",
    "tarea",
    "ejercicio",
    "certamen",
    "laboratorio",
    "entrega", 
    "test",
    "informe",
    "evaluacion"
    ]
presentation_test_dictionary = [
    "gracias",
    "por favor",
    "disculpa",
    "perdon",
    "bienvenido",
    "encantado",
    "amigo",
    "companero",
    "equipo",
    "reunión",
    "evento",
    "saludos",
    "fiesta",
    "invitación",
    "sociedad",
    "familia",
    "grupo",
    "encuentro",
    "hora",
    "hola",
    "instagram"
        ]

In [22]:
# Ejemplo de documentos
testmessage1 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso']
testmessage2 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage3 = ['física, ondas energia gravitación nuclear óptica átom cuántica']
testmessage4 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso. Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage5 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema. La ecuación de Navier-Stokes describe el comportamiento de los fluidos en movimiento. Estas ecuaciones son fundamentales en la dinámica de fluidos y son un conjunto de ecuaciones diferenciales parciales no lineales que describen el flujo de fluidos newtonianos.']

doc1_words = preprocess_document(testmessage1)
doc2_words = preprocess_document(testmessage2)
doc3_words = preprocess_document(testmessage3)
doc4_words = preprocess_document(testmessage4)
doc5_words = preprocess_document(testmessage5)

Ahora vamos a calcular los vectores de cada documento y diccionario con el modelo BERT

In [23]:
# Asegúrate de que doc1_words, doc2_words, presentation_test_dictionary y content_test_dictionary
# sean cadenas de texto o listas de cadenas de texto que representen tus documentos.

# Agregación con BERT
vec_1_avg_bert = make_bert_vec(' '.join(doc1_words), model, tokenizer)
vec_2_avg_bert = make_bert_vec(' '.join(doc2_words), model, tokenizer)
vec_dp_avg_bert = make_bert_vec(' '.join(presentation_test_dictionary), model, tokenizer)
vec_dc_avg_bert = make_bert_vec(' '.join(content_test_dictionary), model, tokenizer)


# Agregación con BERT [CLS]
vec_1_avg_bert_cls = make_bert_cls_vec(' '.join(doc1_words), model, tokenizer)
vec_2_avg_bert_cls = make_bert_cls_vec(' '.join(doc2_words), model, tokenizer)
vec_dp_avg_bert_cls = make_bert_cls_vec(' '.join(presentation_test_dictionary), model, tokenizer)
vec_dc_avg_bert_cls = make_bert_cls_vec(' '.join(content_test_dictionary), model, tokenizer)



In [24]:
print('Similitud texto 1 - Presentacion:',cos_similarity(vec_1_avg_bert, vec_dp_avg_bert),'--','Similitud texto 1 - Contenido:',cos_similarity(vec_1_avg_bert, vec_dc_avg_bert))
print('Similitud texto 2 - Presentacion:',cos_similarity(vec_2_avg_bert, vec_dp_avg_bert),'--','Similitud texto 2 - Contenido:',cos_similarity(vec_2_avg_bert, vec_dc_avg_bert))


Similitud texto 1 - Presentacion: 0.69377226 -- Similitud texto 1 - Contenido: 0.633451
Similitud texto 2 - Presentacion: 0.7217875 -- Similitud texto 2 - Contenido: 0.7161747


In [25]:
print('Similitud texto 1 - Presentacion:',cos_similarity(vec_1_avg_bert_cls, vec_dp_avg_bert_cls),'--','Similitud texto 1 - Contenido:',cos_similarity(vec_1_avg_bert_cls, vec_dc_avg_bert_cls))
print('Similitud texto 2 - Presentacion:',cos_similarity(vec_2_avg_bert_cls, vec_dp_avg_bert_cls),'--','Similitud texto 2 - Contenido:',cos_similarity(vec_2_avg_bert_cls, vec_dc_avg_bert_cls))


Similitud texto 1 - Presentacion: 0.6152363 -- Similitud texto 1 - Contenido: 0.5997947
Similitud texto 2 - Presentacion: 0.6791248 -- Similitud texto 2 - Contenido: 0.6764449


In [26]:
print(doc1_words)
print(doc2_words)

['hola', 'llamar', 'antonio', 'ansioso', 'comenzar', 'curso']
['diagrama', 'cuerpo', 'libre', 'permitir', 'visualizar', 'fuerza', 'actuan', 'sistema']


In [27]:
# Crear "mini-documentos" a partir de los diccionarios
content_mini_doc = ' '.join(content_test_dictionary)
presentation_mini_doc = ' '.join(presentation_test_dictionary)

# Obtener vectores BERT para los "mini-documentos" y documentos de prueba
vec_content = make_bert_cls_vec(content_mini_doc, model, tokenizer)
vec_presentation = make_bert_cls_vec(presentation_mini_doc, model, tokenizer)
vec_test1 = make_bert_cls_vec(' '.join(doc1_words), model, tokenizer)
vec_test2 = make_bert_cls_vec(' '.join(doc2_words), model, tokenizer)

# Calcular la similitud (usando, por ejemplo, la similitud del coseno)
similarity_test1_content = cos_similarity(vec_test1, vec_content)
similarity_test2_content = cos_similarity(vec_test2, vec_content)


In [28]:
print(similarity_test1_content)
print(similarity_test2_content)

0.5997947
0.6764449


Funciona mal!!!! Ahora probamos:

# S-BERT

In [ ]:
# %pip install transformers
# %pip install sentence-transformers
# %pip3 install pytorch torchvision

In [29]:
from sentence_transformers import SentenceTransformer

# Cargar un modelo SBERT en español
model_name = "sentence-transformers/stsb-xlm-r-multilingual" # 
model = SentenceTransformer(model_name)


In [30]:
# Ejemplo de documentos
testmessage1 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso']
testmessage2 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage3 = ['física, ondas energia gravitación nuclear óptica átom cuántica']
testmessage4 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso. Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage5 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema. La ecuación de Navier-Stokes describe el comportamiento de los fluidos en movimiento. Estas ecuaciones son fundamentales en la dinámica de fluidos y son un conjunto de ecuaciones diferenciales parciales no lineales que describen el flujo de fluidos newtonianos.']

doc1_words = preprocess_document(testmessage1)
doc2_words = preprocess_document(testmessage2)
doc3_words = preprocess_document(testmessage3)
doc4_words = preprocess_document(testmessage4)
doc5_words = preprocess_document(testmessage5)

# Obtener vectores de documentos
vec_doc1 = model.encode(' '.join(doc1_words))
vec_doc2 = model.encode(' '.join(doc2_words))
vec_doc3 = model.encode(' '.join(doc3_words))
vec_doc4 = model.encode(' '.join(doc4_words))
vec_doc5 = model.encode(' '.join(doc5_words))

vec_content = model.encode(' '.join(content_test_dictionary))#
vec_presentation = model.encode(' '.join(presentation_test_dictionary))

# Calcular la similitud (usando, por ejemplo, la similitud del coseno)
similarity_test1_content = cos_similarity(vec_doc1, vec_content)
similarity_test2_content = cos_similarity(vec_doc2, vec_content)
similarity_test3_content = cos_similarity(vec_doc3, vec_content)
similarity_test4_content = cos_similarity(vec_doc4, vec_content)
similarity_test5_content = cos_similarity(vec_doc5, vec_content)

similarity_test1_presentacion = cos_similarity(vec_doc1, vec_presentation)
similarity_test2_presentacion = cos_similarity(vec_doc2, vec_presentation)
similarity_test3_presentacion = cos_similarity(vec_doc3, vec_presentation)
similarity_test4_presentacion = cos_similarity(vec_doc4, vec_presentation)
similarity_test5_presentacion = cos_similarity(vec_doc5, vec_presentation)


print(similarity_test1_content)
print(similarity_test2_content)
print(similarity_test3_content)
print(similarity_test4_content)
print(similarity_test5_content)

0.2044222
0.45829985
0.8045501
0.47069472
0.5266916


In [31]:
print(similarity_test1_presentacion)
print(similarity_test2_presentacion)
print(similarity_test3_presentacion)
print(similarity_test4_presentacion)
print(similarity_test5_presentacion)

0.29156387
0.06304032
0.07182588
0.2486218
0.05881759


# Convertir diccionario en frases para S-BERT

En lugar de simplemente unir todas las palabras clave en una sola cadena, podríamos tratar de formar oraciones o frases coherentes que hagan un uso significativo de estas palabras. Esto es porque SBERT está optimizado para entender el significado de oraciones completas en lugar de listas de palabras aisladas.

# STS-BERT

El modelo `stsb-xlm-r-multilingual` es un tipo de modelo de Sentence-BERT (SBERT), específicamente diseñado para tareas de Similitud Semántica de Textos (STS) y es multilingüe. SBERT es una modificación del preentrenado BERT (Bidirectional Encoder Representations from Transformers) para producir incrustaciones (embeddings) de oraciones que pueden ser comparadas eficientemente utilizando la similitud del coseno. Vamos a desglosar las características clave de este modelo:

## Características Principales de `stsb-xlm-r-multilingual`:
### Sentence-BERT (SBERT):

SBERT es una variante de BERT que ha sido modificada y afinada para generar representaciones de oraciones completas, en lugar de tokens individuales. Esto lo hace ideal para tareas donde necesitas comparar la similitud entre oraciones completas o párrafos.

### Optimizado para Tareas STS:

El modelo ha sido afinado específicamente para tareas de Similitud Semántica de Textos, lo que significa que es muy efectivo en medir cuán similares son dos segmentos de texto en términos de su significado.

### Multilingüe:

"XLM-R" se refiere a "Cross-lingual Language Model - RoBERTa", una variante de BERT que ha sido entrenada en varios idiomas. Esto hace que "stsb-xlm-r-multilingual" sea efectivo en trabajar con textos en diversos idiomas, incluyendo, pero no limitándose a, el español.
Aplicaciones:

Este modelo es útil en aplicaciones como la comparación de similitud de oraciones, agrupación semántica, sistemas de recomendación basados en texto y más.

### Rendimiento:

Debido a su entrenamiento especializado y naturaleza multilingüe, este modelo ofrece un equilibrio entre comprensión del lenguaje en varios idiomas y capacidad para realizar tareas de similitud semántica.

### Uso del Modelo:
El modelo puede ser utilizado para codificar textos en vectores de tal manera que textos con significados similares estarán cercanos en el espacio vectorial. Esto es particularmente útil para comparar textos en términos de su contenido semántico, independientemente del idioma en que estén escritos.

Así, `stsb-xlm-r-multilingual` es una herramienta poderosa y versátil para cualquier aplicación que requiera comprender y comparar el significado semántico de textos en múltiples idiomas.

In [32]:
from sentence_transformers import SentenceTransformer, models
import torch

# Cargar el modelo y el tokenizer
# model_name = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

#Modelo optimizado para la tarea de Similitud Semántica de Textos (STS) y es multilingüe, lo cual es útil si estás trabajando con textos en español.
model_name = "sentence-transformers/stsb-xlm-r-multilingual" # 

#Promedio
model = SentenceTransformer(model_name)


In [33]:
# Diccionario de términos clave formado en frases coherentes
content_phrases = [
    "En mecánica cuántica, la función de onda de Schrödinger describe cómo cambian los estados cuánticos con el tiempo.",
    "El principio de incertidumbre de Heisenberg postula límites fundamentales en la precisión de las mediciones simultáneas.",
    "La segunda ley de la termodinámica establece que la entropía total de un sistema aislado siempre aumenta.",
    "La teoría especial de la relatividad de Einstein revoluciona nuestra comprensión de tiempo y espacio con la ecuación E=mc².",
    "La ley de Coulomb describe la fuerza eléctrica entre dos cargas puntuales.",
    "La física de partículas explora el modelo estándar, incluyendo quarks, leptones y bosones de gauge.",
    "La dualidad onda-partícula es un concepto fundamental en la mecánica cuántica, ilustrando la naturaleza dual de la materia.",
    "El electromagnetismo se unifica a través de las ecuaciones de Maxwell, que describen cómo los campos eléctricos y magnéticos se propagan y cómo se generan.",
    "En óptica física, la interferencia y la difracción son fenómenos fundamentales para entender la naturaleza de la luz.",
    "La física nuclear se enfoca en entender las fuerzas que actúan dentro del núcleo atómico, incluyendo la fuerza fuerte y la fisión nuclear.",
    "La cosmología física estudia la estructura y evolución del universo, incluyendo teorías sobre el Big Bang y la energía oscura.",
    "Los experimentos en física de alta energía, como los realizados en el Gran Colisionador de Hadrones, buscan descubrir nuevas partículas.",
    "La mecánica estadística conecta las propiedades macroscópicas de los sistemas con las descripciones microscópicas de sus componentes.",
    "La superconductividad y los fenómenos de estados cuánticos de la materia ofrecen perspectivas para avances tecnológicos."
]


presentation_phrases = [
    "Gracias por su atención y por favor siéntanse libres de hacer preguntas.",
    "Me disculpo de antemano por cualquier error y pido perdón por las molestias.",
    "Bienvenidos a este encuentro donde estaremos encantados de compartir ideas.",
    "Amigos y compañeros, unamos esfuerzos para trabajar en equipo.",
    "Esta reunión será crucial para planificar el próximo evento social.",
    "Saludos a todos, espero que disfruten de la fiesta y la invitación.",
    "La sociedad se beneficia cuando familias y grupos colaboran en encuentros.",
    "La hora de inicio será puntual, así que les digo hola y bienvenidos.",
    "Síganos en Instagram para más actualizaciones sobre futuros eventos."
]


# Codificar las frases del diccionario
vec_content_phrases = [model.encode(phrase) for phrase in content_phrases]
vec_presentacion_phrases = [model.encode(phrase) for phrase in presentation_phrases]


# Ejemplo de documentos
testmessage1 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso']
testmessage2 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage3 = ['física, ondas energia gravitación nuclear óptica átom cuántica']
testmessage4 = ['hola, me llamo antonio y estoy ansioso por comenzar el curso. Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema']
testmessage5 = ['Los diagramas de cuerpo libre nos permiten visualizar que fuerzas actuan sobre un sistema. La ecuación de Navier-Stokes describe el comportamiento de los fluidos en movimiento. Estas ecuaciones son fundamentales en la dinámica de fluidos y son un conjunto de ecuaciones diferenciales parciales no lineales que describen el flujo de fluidos newtonianos.']

doc1_words = preprocess_document(testmessage1)
doc2_words = preprocess_document(testmessage2)
doc3_words = preprocess_document(testmessage3)
doc4_words = preprocess_document(testmessage4)
doc5_words = preprocess_document(testmessage5)

# Obtener vectores de documentos
vec_doc1 = model.encode(' '.join(doc1_words))
vec_doc2 = model.encode(' '.join(doc2_words))
vec_doc3 = model.encode(' '.join(doc3_words))
vec_doc4 = model.encode(' '.join(doc4_words))
vec_doc5 = model.encode(' '.join(doc5_words))

# Calcular la similitud promedio entre el documento y cada frase del diccionario
average_similarity1 = np.mean([cos_similarity(vec_doc1, vec_phrase) for vec_phrase in vec_content_phrases])
average_similarity2 = np.mean([cos_similarity(vec_doc2, vec_phrase) for vec_phrase in vec_content_phrases])
average_similarity3 = np.mean([cos_similarity(vec_doc3, vec_phrase) for vec_phrase in vec_content_phrases])
average_similarity4 = np.mean([cos_similarity(vec_doc4, vec_phrase) for vec_phrase in vec_content_phrases])
average_similarity5 = np.mean([cos_similarity(vec_doc5, vec_phrase) for vec_phrase in vec_content_phrases])

average_similarity1_pres = np.mean([cos_similarity(vec_doc1, vec_phrase) for vec_phrase in vec_presentacion_phrases])
average_similarity2_pres = np.mean([cos_similarity(vec_doc2, vec_phrase) for vec_phrase in vec_presentacion_phrases])
average_similarity3_pres = np.mean([cos_similarity(vec_doc3, vec_phrase) for vec_phrase in vec_presentacion_phrases])
average_similarity4_pres = np.mean([cos_similarity(vec_doc4, vec_phrase) for vec_phrase in vec_presentacion_phrases])
average_similarity5_pres = np.mean([cos_similarity(vec_doc5, vec_phrase) for vec_phrase in vec_presentacion_phrases])


print('Contenido')
print(average_similarity1)
print(average_similarity2)
print(average_similarity3)
print(average_similarity4)
print(average_similarity5)
print('Presentacion')
print(average_similarity1_pres)
print(average_similarity2_pres)
print(average_similarity3_pres)
print(average_similarity4_pres)
print(average_similarity5_pres)

Contenido
0.10894067
0.39935452
0.5988185
0.31115535
0.48187786
Presentacion
0.28767073
0.24628721
0.074146956
0.29530466
0.12177021
